In [8]:
from pymongo import MongoClient

tahun = 2023

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Sesuaikan dengan URI MongoDB Anda
db = client['youtube_rewind_indonesia']  # Ganti dengan nama database Anda

# Koleksi
labeling_2015 = db[f'labeling_{tahun}']
preprocessing_new_2015 = db[f'preprocessing_new_{tahun}']
labeling_new_2015 = db[f'labeling_new_{tahun}']

# Mengambil semua data dari labeling_tahun
labeling_data = list(labeling_2015.find())

for label in labeling_data:
    text_original = label['textOriginal']
    
    # Mencari data yang sesuai di preprocessing_new_tahun
    preprocessed_data = preprocessing_new_2015.find_one({'textOriginal': text_original})
    
    if preprocessed_data:
        # Menggabungkan data dan mempertahankan _id dari preprocessing_new_tahun
        merged_data = {**preprocessed_data, **label}
        
        # Memasukkan data yang sudah digabungkan ke labeling_new_tahun
        labeling_new_2015.insert_one(merged_data)

print("Proses penggabungan selesai.")


Proses penggabungan selesai.
